# Test Model

In [1]:
import os
from pathlib import Path
import sys
sys.path.append(str(Path(os.getcwd()).parent))

from settings.global_settings import GlobalSettings

config = GlobalSettings.get_config(
    config_file = "../config.ini",
    secrets_file = "../secrets.ini"
)
from dataset.video_loader import VideoDataLoader
from dataset.video_dataset import VideoDataset
from model.training_loop import train
from torch.utils.data import random_split
from model.multimodal_har_model import MultiModalHARModel

2025-10-20 10:25:53,563 - INFO - Sentry DSN set to: https://f4f21cc936b3ba9f5dbc1464b7a40ea4@o4504168838070272.ingest.us.sentry.io/4506464560414720
2025-10-20 10:25:53,564 - INFO - Sentry initialized with environment: development


Loading config...
Loading secrets...


## Initializing Training

**Creating Dataloaders**

In [2]:
video_data_loader = VideoDataLoader(
    path=config.model_settings.video_data_dir
)

video_dataset = VideoDataset(
    video_data_loader=video_data_loader,
    normalization_type="across_frames",
)

len(video_dataset)
for _ in video_dataset:
    pass
len(video_dataset.labels_map)

2025-10-20 10:25:55,288 - INFO - [VideoDataLoader] Loding action videos for action: a01
2025-10-20 10:25:55,612 - INFO - [VideoDataLoader] Loding action videos for action: a02
2025-10-20 10:25:55,970 - INFO - [VideoDataLoader] Loding action videos for action: a03
2025-10-20 10:25:56,216 - INFO - [VideoDataLoader] Loding action videos for action: a04
2025-10-20 10:25:56,642 - INFO - [VideoDataLoader] Loding action videos for action: a05
2025-10-20 10:25:57,128 - INFO - [VideoDataLoader] Loding action videos for action: a06
2025-10-20 10:25:57,466 - INFO - [VideoDataLoader] Loding action videos for action: a08
2025-10-20 10:25:58,159 - INFO - [VideoDataLoader] Loding action videos for action: a09
2025-10-20 10:25:58,761 - INFO - [VideoDataLoader] Loding action videos for action: a11
2025-10-20 10:25:58,993 - INFO - [VideoDataLoader] Loding action videos for action: a12


10

**Splitting Train and Test Datasets**

In [3]:
num_total = len(video_dataset)
num_train = int(0.8 * num_total)
num_test = num_total - num_train
train_dataset, test_dataset = random_split(video_dataset, [num_train, num_test])

**Creating Model**

In [4]:

har_model = MultiModalHARModel(
    obj_in=train_dataset[0].graphs_objects[0].x.shape[1],
    joint_in=train_dataset[0].graphs_joints[0].x.shape[1],
    gat_hidden=128,
    gat_out=128,
    temporal_hidden=128,
    num_classes=len(video_dataset.labels_map), 
    dropout=0.5,
)

## Training Model

In [5]:
train_history = train(
    model=har_model,
    video_dataset=train_dataset,
    device='mps',
    epochs=25,
)

2025-10-20 10:26:04,440 - INFO - Starting training loop...
Epoch 1/25: 100%|██████████| 1180/1180 [06:20<00:00,  3.10it/s]
2025-10-20 10:32:24,781 - INFO - Epoch 1/25, Loss: 2.0124
Epoch 2/25: 100%|██████████| 1180/1180 [06:07<00:00,  3.21it/s]
2025-10-20 10:38:31,830 - INFO - Epoch 2/25, Loss: 1.3966
Epoch 3/25: 100%|██████████| 1180/1180 [06:07<00:00,  3.21it/s]
2025-10-20 10:44:38,884 - INFO - Epoch 3/25, Loss: 1.0965
Epoch 4/25: 100%|██████████| 1180/1180 [06:09<00:00,  3.20it/s]
2025-10-20 10:50:47,955 - INFO - Epoch 4/25, Loss: 0.9716
Epoch 5/25: 100%|██████████| 1180/1180 [06:08<00:00,  3.20it/s]
2025-10-20 10:56:56,270 - INFO - Epoch 5/25, Loss: 0.8799
Epoch 6/25: 100%|██████████| 1180/1180 [06:14<00:00,  3.15it/s]
2025-10-20 11:03:10,654 - INFO - Epoch 6/25, Loss: 0.8144
Epoch 7/25: 100%|██████████| 1180/1180 [06:09<00:00,  3.20it/s]
2025-10-20 11:09:19,835 - INFO - Epoch 7/25, Loss: 0.7349
Epoch 8/25: 100%|██████████| 1180/1180 [06:08<00:00,  3.20it/s]
2025-10-20 11:15:28,529

**Saving Model**

In [6]:
har_model.save(
    training_history=train_history
)

2025-10-20 13:04:09,168 - INFO - Saving model to /Volumes/KODAK/masters/model/validation_datasets/NW-UCLA/model/har_model_v0.0.0_nw_ucla_2025-10-20 13:04:09.166170.pht...
2025-10-20 13:04:09,352 - INFO - Model saved successfully.


In [7]:
test_video_indices = test_dataset.indices
print("Test video IDs:", test_video_indices)

Test video IDs: [6, 1012, 401, 306, 246, 301, 853, 1106, 194, 113, 599, 1438, 138, 1318, 423, 422, 338, 639, 765, 922, 1322, 304, 379, 1380, 319, 1429, 1098, 1185, 1154, 905, 159, 1445, 993, 419, 45, 322, 1426, 830, 1138, 767, 1285, 184, 449, 728, 938, 368, 697, 281, 1284, 1189, 1104, 314, 1259, 160, 1025, 1312, 1363, 1466, 458, 877, 93, 456, 34, 1294, 466, 638, 390, 400, 1046, 1348, 230, 953, 809, 642, 706, 886, 1417, 975, 806, 1136, 154, 546, 766, 1395, 776, 1033, 841, 1151, 1183, 1276, 1323, 937, 569, 276, 217, 1082, 995, 557, 1017, 1112, 994, 876, 1239, 1415, 907, 356, 441, 66, 1263, 884, 343, 1393, 515, 64, 158, 420, 127, 0, 1308, 1277, 935, 416, 87, 185, 227, 942, 308, 1178, 926, 397, 122, 1462, 79, 1044, 1014, 636, 470, 1399, 1093, 1186, 1134, 606, 142, 210, 398, 947, 669, 843, 1193, 524, 335, 793, 1152, 403, 990, 825, 717, 597, 751, 1262, 39, 90, 378, 448, 225, 603, 883, 1019, 1188, 970, 1304, 665, 1147, 1076, 514, 564, 572, 959, 18, 58, 490, 1269, 1260, 1329, 346, 199, 949, 59

In [8]:
import torch
from datetime import datetime

model_settings = config.model_settings
save_dir = "~/masters-models"
os.makedirs(save_dir, exist_ok=True)

save_path = os.path.join(
    save_dir,
    f"har_model_{model_settings.model_version}_{model_settings.dataset_prefix}_100perc_{datetime.now()}.pht"
)
torch.save({
"model_state_dict": har_model.state_dict(),
"training_history": train_history,
"model_config": har_model._model_config,
"test_set_indices": test_video_indices,
}, save_path)


## Running tests

**Accuracy**

In [9]:
def evaluate(model, dataset, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for i in range(len(dataset)):
            sample = dataset[i]
            label = sample.label.to(device)

            # Move all graph tensors to device
            graphs_objects = [g.to(device) for g in sample.graphs_objects]
            graphs_joints = [g.to(device) for g in sample.graphs_joints]

            # Forward pass
            output = model(graphs_objects, graphs_joints)

            # Compute prediction
            if output.dim() == 1:
                predicted = torch.argmax(output).unsqueeze(0)
            else:
                _, predicted = torch.max(output, dim=1)

            correct += (predicted == label).sum().item()
            total += 1

    accuracy = 100 * correct / total
    return accuracy


In [10]:
accuracy_evaluation = evaluate(har_model, test_dataset, "mps")
print(f"Test Accuracy: {accuracy_evaluation:.2f}%")

Test Accuracy: 63.73%
